# **Linearly Parametrized Dynamics and Inertia Representations**

This document provides a concise overview of linear parametrization of dynamics and the calculation of body and joint regressors.

<!-- It is designed to serve as a handy reference to accompany our [mujoco-sysid repo](https://github.com/lvjonok/mujoco-sysid) and the associated demo notebook <a href="https://colab.research.google.com/github/lvjonok/mujoco-sysid/blob/master/examples/mujoco_sysid_demo.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="90" align="center"/></a>. -->

The background theory is well-established and thoroughly explained in the following seminal papers:

* [Identification of Fully Physical Consistent Inertial Parameters using Optimization on Manifolds](https://arxiv.org/pdf/1610.08703.pdf)
* [On the closed form computation of the dynamic matrices and their differentiations](https://ieeexplore.ieee.org/document/6696688)
* [Linear Matrix Inequalities for Physically-Consistent Inertial Parameter Identification: A Statistical Perspective on the Mass Distribution](https://arxiv.org/abs/1701.04395)
* [Smooth Parameterization of Rigid-Body Inertia](https://ieeexplore.ieee.org/document/9690029)

This notebook will be regularly updated along with the main repo. We encourage you to revisit periodically to stay informed about the most recent updates.
<!-- The next update is planned for mid-May and will focus on various inertia representations and physical consistency constraints. -->

But first, let's start with the fundamentals. To simplify some calculations, we will exploit the symbolic capabilities of Python via the `sympy` library:


In [ ]:
import sympy as sp

## Notation and Preliminaries



Before proceeding further, let us introduce some notation and recall the basic concepts. If you are not familiar with some of these, please refer to Roy Featherstone's excellent book ["Rigid Body Dynamics Algorithms"](https://link.springer.com/book/10.1007/978-1-4899-7560-7).



- Given $u, v \in \mathbb{R}^{3}, S(u) \in \mathbb{R}^{3 \times 3}$ denotes the skew-symmetric matrix-valued operator associated with the cross product in $\mathbb{R}^{3}$, such that $S(u) v=u \times v$

In [ ]:
def S(u):
    return sp.Matrix([[0, -u[2], u[1]],
                      [u[2], 0, -u[0]],
                      [-u[1], u[0], 0]])



- $W$ denotes an world (inertial) frame and $B$ a local (body-fixed) frame .

- $p_{B} \in \mathbb{R}^{3}$ denotes the origin of the $B$ frame expressed in the inertial frame, while ${ }^{W} R_{B} \in S O(3)$ or simply $R$ is the rotation matrix that transforms a $3 \mathrm{D}$ vector expressed with the orientation of the $B$ frame in a 3D vector expressed in the $W$ frame. $\omega \in \mathbb{R}^{3}$ denotes the body angular velocity of body $B$, defined as $S(\omega)=R^{\top} \dot{R}$.

- $\mathrm{v} \in \mathbb{R}^{6}$ indicates the body twist (spatial velocity), i.e :

$$
\mathrm{v}=\left[\begin{array}{l} v \\  
\omega
\end{array}\right]
$$

the spatial linear velocity $v =  R^{\top}\dot{p}_B$

- $\mathrm{f} \in \mathbb{R}^{6}$ indicates an external wrench exerted on the body i.e:

$$
\mathrm{f}=\left[\begin{array}{l}
f \\
\tau  
\end{array}\right]
$$

where $f$ is the 3D  force and $\tau$ the 3D torque, that are expressed in frame $B$.

- Given $\mathrm{v}=\left[\begin{array}{ll}v^{\top} & \omega^{\top}\end{array}\right]^{\top} \in \mathbb{R}^{6}, \mathrm{v} \hat{\times}$ is the $6 \mathrm{D}$ force cross product operator, defined as:

$$
\mathrm{v} \hat{\times}=\left[\begin{array}{cc}
S(\omega) & 0_{3 \times 3} \\
S(v) & S(\omega)  
\end{array}\right]  
$$


In [ ]:
def motion_cross(v):
    v_lin = v[:3]
    omega = v[3:]
    return sp.BlockMatrix([[S(omega), sp.zeros(3, 3)],
                           [S(v_lin), S(omega)]])

<!-- - Given a symmetric matrix $I \in \mathbb{R}^{3 \times 3}$ the vech operator denotes the serialization operation on symmetric matrices:

$$
\operatorname{vech}\left(\left[\begin{array}{lll}
I_{x x} & I_{x y} & I_{x z} \\
I_{x y} & I_{y y} & I_{y z} \\  
I_{x z} & I_{y z} & I_{z z}
\end{array}\right]\right)=\left[\begin{array}{c}
I_{x x} \\
I_{x y} \\
I_{x z} \\
I_{y y} \\  
I_{y z} \\
I_{z z}  
\end{array}\right]
$$

- Given a symmetric matrix $I \in \mathbb{R}^{n \times n}, I \succeq 0$ denotes that the matrix is positive semidefinite, i.e. that all its eigenvalues are nonnegative. -->

- $g \in \mathbb{R}^{3}$ is the constant vector of gravity acceleration in inertial frame $W$.

- $\mathrm{a}^{g} \in \mathbb{R}^{6}$ denotes the proper body acceleration, i.e. the difference between the body **spatial acceleration** and the gravity acceleration:  

$$
\mathrm{a}_{g}=\dot{\mathrm{v}}-\left[\begin{array}{c}
 R^{\top} g \\
0_{3}  
\end{array}\right]
$$

**Note:** Body acceleration is spatial **NOT** the classical one which is given by:
$$
a = \dot{\mathrm{v}} + S(\omega)\mathrm{v}
$$



## **Single Rigid Body Dynamics**

Using the above notation, we can formulate the Newton-Euler equations for a single rigid body (in the local frame) as:

$$
\begin{equation*}
M \mathrm{a}_{g}+\mathrm{v} \hat{\times} M \mathrm{v}=\mathrm{f}
\end{equation*}  
$$

where $M \in \mathbb{R}^{6 \times 6}$ is the $6 \mathrm{D}$ inertia matrix (also known as spatial inertia):

$$
M=\left[\begin{array}{cc}
m 1_{3} & -m S(r_c) \\
m S(r_c) & I_{B}  
\end{array}\right]
$$



In [ ]:
def M(m, h, I_B):
    I_3 = sp.eye(3)
    return sp.BlockMatrix([[m*I_3, -S(h)],
                           [S(h), I_B]])

The terms in the inertia matrix are:
- $m \in \mathbb{R}$ is the mass of the rigid body.  
- $h = mr_c \in \mathbb{R}^{3}$ is the so-called first inertia moment, i.e., the product of mass $m$ and the center of mass of the rigid body $r_c$, expressed in frame $B$.
- $I \in \mathbb{R}^{3 \times 3}$ is the $3 \mathrm{D}$ inertia tensor of the rigid body, expressed with the orientation of frame $B$ and with respect to the frame $B$ origin.

$$
\mathbf{I} =\left[\begin{array}{lll}
I_{x x} & I_{x y} & I_{x z} \\
I_{x y} & I_{y y} & I_{y z} \\
I_{x z} & I_{y z} & I_{z z}
\end{array}\right]
$$

The inertia tensor is represented in the body-fixed frame. Due to the symmetry of the inertia tensor, there are only $6$ unique parameters.

It is not difficult to see that the positive definite symmetric spatial inertia matrix $M$ completely determines the inertial properties of a single rigid body. Furthermore, in the Newton-Euler equations above, there are only linear operations (matrix multiplications) with respect to $M$. This implies that the force (inverse dynamics) is **linearly parametrizable** with respect to the elements of $M$..

## **Linear Parametrization**

### **Inverse Dynamics**

As we have deduced above, conceptually the inverse dynamics is linearly parametrizable:

$$
M \mathrm{a}_{g}+\mathrm{v} \hat{\times} M \mathrm{v} = \mathbf{Y}(\mathrm{v}, \mathrm{a}_{g})\boldsymbol{\theta} =\mathrm{f}
$$

where $\mathbf{Y}(\bar{v}, \mathrm{a}^{g}) \in \mathbb{R}^{6\times10}$ is the so-called **body regressor**.

The vector $\theta \in \mathbb{R}^{10}$ represents the unique parameters of the spatial inertia matrix:

$$
\mathbf{\theta} = \begin{bmatrix} m, h, \operatorname {vech}(I)\end{bmatrix}^T  = \begin{bmatrix} m, mr_x, mr_y, mr_z, I_{xx}, I_{xy}, I_{yy}, I_{xz}, I_{yz}, I_{zz} \end{bmatrix}^T
$$

Here $\operatorname {vech}$ is the linear [half-vectorization operator ](https://en.wikipedia.org/wiki/Vectorization_(mathematics)#Half-vectorization) that maps components of a symmetric matrix to a vector. This operator takes advantage of the symmetry of the inertia matrix to represent it compactly using only its unique elements.

Although these ten parameters are sufficient to fully characterize a rigid body's inertial properties, they may not be the most physically meaningful or intuitive set. Recent work has shown these standard inertial parameters are [tightly coupled with the density distribution](https://arxiv.org/pdf/1701.04395) of the rigid body. More insightful parametrizations that relate directly to the mass distribution may be preferable. However, we will begin with the standard parameters as they are the most commonly used and easy to work with mathematically. Later, we will discuss alternative inertial parametrizations as well as important physical consistency constraints they should obey.

The most direct way to obtain the body regressor matrix is to take the partial derivative of the force $\mathrm{f}$ with respect to the parameters $\boldsymbol{\theta}$. Alternatively, one could carefully reparametrize the cross product terms in the ID. Here we will use symbolic differentiation for clarity.

First, let us create a function to calculate the inverse dynamics and a helper function to construct the spatial inertia from the parameter vector:

In [ ]:
def force(a_g, v, M):
    return sp.Matrix(M@a_g + motion_cross(v)@M@v)

def parameters2spatial(params):
    m, mr_x, mr_y, mr_z, I_xx, I_xy, I_yy, I_xz, I_yz, I_zz = params
    h = sp.Matrix([mr_x, mr_y, mr_z])
    I_B = sp.Matrix([[I_xx, I_xy, I_xz],
                    [I_xy, I_yy, I_yz],
                    [I_xz, I_yz, I_zz]])
    return M(m, h, I_B)

Now let us calculate the symbolic expression for the inverse dynamics:

In [ ]:
# Define symbolic variables for the proper body acceleration and body twist
a_g = sp.Matrix(sp.symbols('a.vel[1] a.vel[2] a.vel[3] a.ang[1] a.ang[2] a.ang[3]'))
v = sp.Matrix(sp.symbols('v.vel[1] v.vel[2] v.vel[3] v.ang[1] v.ang[2] v.ang[3]'))
# The sequence of inertial parameters matches the order of the half-vectorization
params = sp.Matrix(sp.symbols('m mr_x mr_y mr_z I_xx I_xy I_yy I_xz I_yz I_zz'))

force_sym = force(a_g, v, parameters2spatial(params))
force_sym

Matrix([
[                                                                                                                                                                                                                                       a.ang[2]*mr_z - a.ang[3]*mr_y + a.vel[1]*m + m*v.ang[2]*v.vel[3] - m*v.ang[3]*v.vel[2] - mr_x*v.ang[2]**2 - mr_x*v.ang[3]**2 + v.ang[1]*(mr_y*v.ang[2] + mr_z*v.ang[3])],
[                                                                                                                                                                                                                                      -a.ang[1]*mr_z + a.ang[3]*mr_x + a.vel[2]*m - m*v.ang[1]*v.vel[3] + m*v.ang[3]*v.vel[1] - mr_y*v.ang[1]**2 - mr_y*v.ang[3]**2 + v.ang[2]*(mr_x*v.ang[1] + mr_z*v.ang[3])],
[                                                                                                                                                                                          

To get the symbolic expression for the body regressor, we take the Jacobian of the inverse dynamics with respect to the parameter vector $\theta$:

In [ ]:
Ysym = force_sym.jacobian(params)
print(Ysym)

Matrix([[a.vel[1] + v.ang[2]*v.vel[3] - v.ang[3]*v.vel[2], -v.ang[2]**2 - v.ang[3]**2, -a.ang[3] + v.ang[1]*v.ang[2], a.ang[2] + v.ang[1]*v.ang[3], 0, 0, 0, 0, 0, 0], [a.vel[2] - v.ang[1]*v.vel[3] + v.ang[3]*v.vel[1], a.ang[3] + v.ang[1]*v.ang[2], -v.ang[1]**2 - v.ang[3]**2, -a.ang[1] + v.ang[2]*v.ang[3], 0, 0, 0, 0, 0, 0], [a.vel[3] + v.ang[1]*v.vel[2] - v.ang[2]*v.vel[1], -a.ang[2] + v.ang[1]*v.ang[3], a.ang[1] + v.ang[2]*v.ang[3], -v.ang[1]**2 - v.ang[2]**2, 0, 0, 0, 0, 0, 0], [0, 0, a.vel[3] + v.ang[1]*v.vel[2] - v.ang[2]*v.vel[1], -a.vel[2] + v.ang[1]*v.vel[3] - v.ang[3]*v.vel[1], a.ang[1], a.ang[2] - v.ang[1]*v.ang[3], -v.ang[2]*v.ang[3], a.ang[3] + v.ang[1]*v.ang[2], v.ang[2]**2 - v.ang[3]**2, v.ang[2]*v.ang[3]], [0, -a.vel[3] - v.ang[1]*v.vel[2] + v.ang[2]*v.vel[1], 0, a.vel[1] + v.ang[2]*v.vel[3] - v.ang[3]*v.vel[2], v.ang[1]*v.ang[3], a.ang[1] + v.ang[2]*v.ang[3], a.ang[2], -v.ang[1]**2 + v.ang[3]**2, a.ang[3] - v.ang[1]*v.ang[2], -v.ang[1]*v.ang[3]], [0, a.vel[2] - v.ang[1]*

*The* resulting matrix is the **body regressor**, which provides a linear parametrization of the inverse dynamics for a single rigid body, assuming the applied force and torque are known. However, in practice, robotic systems consist of multiple interconnected rigid bodies, and the spatial forces acting between bodies are internal to the system and not directly measurable while the only available measurements come from sensors located at the joints. This means the generalized coordinates used to describe the system are the joint angles and velocities. To make the inverse dynamics useful for control and identification, we need to reformulate it to be linear in the parameters while using joint-based generalized coordinates.



**Joint Regressor**

In the joint regressor formulation, we seek a parametrization of the multi-body system whose joint space inverse dynamics can be written in the form:

$$
\begin{equation*}
\mathbf{M}(\mathbf{q}) \dot{\boldsymbol{\nu}}+\mathbf{c}(\mathbf{q}, \boldsymbol{\nu})=\mathbf{Q} = \mathbf{Y}(\mathbf{q}, \boldsymbol{\nu}, \dot{\boldsymbol{\nu}}) \boldsymbol{\theta}
\end{equation*}
$$

where:
* $nv \in \mathbb{N}_{+}$ is the number of degrees of freedom  
* $\mathbf{M} \in \mathbb{R}^{nv \times nv}$ is the configuration-dependent positive definite inertia matrix
* $\mathbf{q} \in \mathcal{Q}$ is the configuration, with $\mathcal{Q}$ being the configuration manifold
* $\boldsymbol{\nu} \in \mathbb{R}^{nv}$ is the generalized velocity
* $\mathbf{c} \in \mathbb{R}^{nv}$ represents the combined effect of Coriolis, gravity, and other forces
* $\mathbf{Q} \in \mathbb{R}^{nv}$ is the vector of generalized forces (usually control inputs and contact forces)
* $nb \in \mathbb{N}_{+}$ is the number of bodies in the system
* $\mathbf{Y} \in \mathbb{R}^{nv \times 10nb}$ is the whole-system joint regressor matrix
* $\boldsymbol{\theta}^{\top} = [\boldsymbol{\theta}^{\top}_1, \cdots ,\boldsymbol{\theta}^{\top}_{nb}]^{\top} \in \mathbb{R}^{10nb}$ is the vector of stacked inertial parameters for each body

The question is how to calculate the matrix $\mathbf{Y}$ based only on "joint" signals. There are several ways to do this, but one of the easiest and most effective methods is based on the body regressor defined above.

Let us focus on a single body whose velocity and acceleration are given by:

$$
\mathrm{v}_j(q, \nu) = J_j(q)\nu
$$

$$
\mathrm{a}_{g_j}(q, \nu, \dot{\nu})=J_j(q)\dot{\nu} + \frac{dJ_j}{dt}\nu-\left[\begin{array}{c}
R(q)^{\top} g \\
0_{3}  
\end{array}\right]
$$

where $J_j$ is the Jacobian matrix that maps the generalized velocities $\nu$ into the spatial velocity of the $j$-th body frame origin expressed in the local frame. It is worth noting that the Jacobians and all related quantities are calculated based on forward kinematics (usually done by the forward recursion of the Recursive Newton-Euler Algorithm), and do not require any reasoning about dynamics.

Given this notation, we may define the generalized force originating from the spatial twist at the origin of the $j$-th body as follows:

$$
Q_j = J^{\top}_j \mathrm{f}_j
$$

Substituting the expression for $\mathrm{f}_j$ parametrized by the body regressor and taking into account the expressions for velocity and acceleration yields:

$$
J^{\top}_j \mathbf{Y}_{b_j}\Big(\mathrm{v}_j(q, \nu), \mathrm{a}_{g_j}(q, \nu, \dot{\nu})\Big)\boldsymbol{\theta}_j = J^{\top}_j \mathbf{Y}_{b_j}(q, \nu, \dot{\nu})\boldsymbol{\theta}_j = \mathbf{Y}_{j}(q, \nu, \dot{\nu})\boldsymbol{\theta}_j
$$

Summing up the contributions from each generalized force, we obtain:

$$
\mathbf{M}(\mathbf{q}) \dot{\boldsymbol{\nu}}+\mathbf{c}(\mathbf{q}, \boldsymbol{\nu}) = \mathbf{Q} = \sum^{nb}_{j=1} Q_j  = \sum^{nb}_{j=1}\mathbf{Y}_{j}(q, \nu, \dot{\nu})\boldsymbol{\theta}_j  = \mathbf{Y}(q, \nu, \dot{\nu})\boldsymbol{\theta}
$$

Thus, one can formulate the system joint regressor as:

$$
\mathbf{Y}(q, \nu, \dot{\nu}) =
\begin{bmatrix}
J_1^{\top} Y_{b_1}\big(\mathrm{v}_1(q, \nu), \mathrm{a}_{g_1}(q, \nu, \dot{\nu})\big) & \ldots & J_{nb}^{\top}Y_{b_{nb}}\big(\mathrm{v}_{nb}(q, \nu), \mathrm{a}_{g_{nb}}(q, \nu, \dot{\nu})\big) \end{bmatrix}
$$

which consists of the body regressors with their respective velocity and acceleration expressions substituted and mapped to joint torques via the Jacobian matrices.

Some modeling libraries provide functions to compute these regressors.

<!-- We also provide some [handy functions to calculate body and joint regressors in MuJoCo](https://github.com/lvjonok/mujoco-sysid/blob/master/mujoco_sysid/modeling.py). -->

## **Inertia Parametrization**

In the discussion above, we have defined the inertial parameters of a rigid body as the following vector in $\mathbb{R}^{10}$:

$$
\boldsymbol{\theta}=\left[m, h_x, h_y, h_z, I_{x x}, I_{x y}, I_{y y}, I_{x z}, I_{y z}, I_{z z}\right]
$$

This parametrization is intuitive and straightforward. However, it might seem that the parameters $\boldsymbol{\theta}$ are independent variables, but this is not entirely the case.

To clarify this, let us redefine the inertial parameters through the density distribution $\rho(\mathbf{x})$:
$$
\begin{aligned}
m & =\int_{\mathbb{R}^3} \rho(\mathbf{x}) \, \mathrm{d} V \\
\mathbf{h} & =\int_{\mathbb{R}^3} \mathbf{x} \rho(\mathbf{x}) \, \mathrm{d} V=\left[\begin{array}{lll}
h_x & h_y & h_z
\end{array}\right]^{\top} \\
\mathbf{I} & =\iiint_{\mathbb{R}^3}\left[\begin{array}{ccc}
y^2+z^2 & -x y & -x z \\
-x y & x^2+z^2 & -y z \\
-x z & -y z & x^2+y^2
\end{array}\right] \rho(\mathbf{x}) \mathrm{d} V \\
& =\left[\begin{array}{lll}
I_{x x} & I_{x y} & I_{x z} \\
I_{x y} & I_{y y} & I_{y z} \\
I_{x z} & I_{y z} & I_{z z}
\end{array}\right]
\end{aligned}
$$

Thus, for the inertia vector $\boldsymbol{\theta}$ to maintain **physical consistency**, all parameters must originate from the same density distribution function, i.e:

---

Candidate inertial parameters $\boldsymbol{\theta}$ are fully physically consistent if and only if there exists some finite-valued density field $\rho(\cdot): \mathbb{R}^{3} \mapsto \mathbb{R}_{\geq 0}$ such that $\boldsymbol{\theta}$ satisfies integral equalities above and $m>0$.

---

This requirement introduces specific constraints on $\boldsymbol{\theta}$. By considering physical consistency, improvements can be made in both the accuracy and efficiency of parameter estimation, leading to enhancements in adaptive algorithms.

Research in this field has gained momentum recently, leading to the development of various conditions and **inertia parametrizations** that guarantee physical consistency. The above we have discussing the different inertia parametrization, this text is adopted from recent paper on smooth inertia parametrization.


### **Eigen Decomposition**

It is known that every rigid body has a principal frame in which the inertia tensor can be diagonalized, i.e. represented via eigen decomposition as:

$$
\mathbf{I} = \mathbf{R} \mathbf{D} \mathbf{R}^\top
$$

Here, $\mathbf{D} = \operatorname{diag}(D_x, D_y, D_z)$ consists of the principal moments of inertia, and $\mathbf{R}$ is a rotation matrix belonging to $\mathrm{SO}(3)$, the special orthogonal group.

[Traversaro et al.](https://arxiv.org/pdf/1610.08703), demonstrated that this principal basis can be effectively utilized to parametrize a set of inertial parameters that are fully physically consistent. Specifically, a set of parameters $\theta$ is considered fully physically consistent if $\mathbf{I} \succ 0$, $m > 0$, and the following triangular inequalities are satisfied:

$$
D_x < D_y + D_z, \quad D_y < D_x + D_z, \quad D_z < D_x + D_y
$$

These inequalities stem from the relationships among the principal moments and the second moments $L_x, L_y, L_z$ of the rigid body along its principal axes, given by:

$$
D_x = L_y + L_z, \quad D_y = L_x + L_z, \quad D_z = L_x + L_y
$$

where

$$
\begin{aligned}
L_x &= \int_{\mathbb{R}^3} x'^2 \rho(\mathbf{x}) \, dV, \\
L_y &= \int_{\mathbb{R}^3} y'^2 \rho(\mathbf{x}) \, dV, \\
L_z &= \int_{\mathbb{R}^3} z'^2 \rho(\mathbf{x}) \, dV,
\end{aligned}
$$

and the vector $\mathbf{x}' = [x', y', z']^\top$ represents the coordinates transformed by $\mathbf{R}$, such that $\mathbf{R} \mathbf{x}' = \mathbf{x}$. The positivity of $L_x, L_y, L_z$ implies the validity of the triangular inequalities.

This leads to a natural parametrization of the inertial parameters $\theta$ in terms of a product space $\mathbb{R}_{>0} \times \mathbb{R}^3 \times \mathrm{SO}(3) \times \mathbb{R}_{>0}^3$, using:

$$
m \in \mathbb{R}_{>0}, \quad \mathbf{h} \in \mathbb{R}^3, \quad \mathbf{R} \in \mathrm{SO}(3), \quad \mathbf{L} = \begin{bmatrix} L_x \\ L_y \\ L_z \end{bmatrix} \in \mathbb{R}_{>0}^3
$$

The resultant rotational inertia $\mathbf{I}$ and the diagonal matrix $\mathbf{D}$ are derived from $\mathbf{L}$. However, this mapping's domain is not a vector space due to the involvement of manifolds like $\mathrm{SO}(3)$ and $\mathbb{R}_{>0}^3$. Therefore, physically-consistent inertial parameter estimation often requires methods such as projections or Riemannian optimization. An alternative could involve reparameterizing $\mathrm{SO}(3)$ using exponential coordinates $\boldsymbol{\beta} \in \mathbb{R}^3$ as $\mathbf{R} = e^{\mathbf{S}(\boldsymbol{\beta})}$.

---

Thus the set of physically consistant parameterts can be derived via:

$$
\boldsymbol{\psi} = [m \in \mathbb{R}_{>0}, \mathbf{h} \in \mathbb{R}^3, \boldsymbol{\beta} \in \mathbb{R}^3, \mathbf{L} \in \mathbb{R}_{>0}^3]
$$

---

Despite the theoretical elegance, this eigen-decomposition-based parametrization encounters singularities, identified in [related research](https://ieeexplore.ieee.org/document/9690029). These singularities occur when two of the second moments are equal i.e when the body exhibits rotational symmetries, which are common in robotic links. This will result in singular  Jacobian:

$$\boldsymbol{J}_\psi = \frac{\partial \theta}{\partial \psi}$$

These singularities can significantly affect differentiable physics applications and online adaptive estimation. Moreover, the parametrization is not bijective, as multiple parameter sets can yield the same inertia matrix. These challenges necessitate the exploration of alternative representations for physically consistent inertial parameters.

### **Pseudo Inertia**

From our earlier discussions, it might appear that a set of physically consistent parameters cannot be represented in a vector space format, particularly due to the involvement of $\mathrm{SO}(3)$. One might also think that any parameterization using $\mathbb{R}^{10}$ would encounter similar issues. However, recent research challenges this assumption. Investigations into two specific parameterizations have demonstrated a one-to-one mapping from the vector space of $\theta$ to a set of physically consistent parameters, each devoid of singularities. Both approaches incorporate the concept of a [$4 \times 4$ pseudoinertia matrix](https://arxiv.org/abs/1701.04395), defined as follows:

$$
\mathbf{J}(\boldsymbol{\theta}) = \left[\begin{array}{cc}
\boldsymbol{\Sigma} & \mathbf{h} \\
\mathbf{h}^\top & m
\end{array}\right]
$$

where:

$$
\boldsymbol{\Sigma} = \frac{1}{2} \operatorname{Tr}(\mathbf{I}) \mathbf{1}_{3} - \mathbf{I} \quad \text{and} \quad \mathbf{I} = \operatorname{Tr}(\boldsymbol{\Sigma}) \mathbf{1}_{3} - \boldsymbol{\Sigma}
$$


Let us create the function to facilitate these calculations:

In [ ]:
def theta2pseudo(theta):
    """
    Converts theta parameters into the pseudo inertia matrix.

    Args:
        theta (np.ndarray): Contains mass, first moments, and inertia tensor components:
            [m, h_x, h_y, h_z, I_xx, I_xy, I_yy, I_xz, I_yz, I_zz]
    Returns:
        np.ndarray: Pseudo inertia matrix.
    """
    m = theta[0]
    h = theta[1:4]
    I_xx, I_xy, I_yy, I_xz, I_yz, I_zz = theta[4:]

    I_bar = sp.Matrix([[I_xx, I_xy, I_xz], [I_xy, I_yy, I_yz], [I_xz, I_yz, I_zz]])

    Sigma = 0.5 * sp.trace(I_bar) * sp.eye(3) - I_bar

    pseudo_inertia = sp.Matrix(sp.zeros(4))
    pseudo_inertia[:3, :3] = Sigma
    pseudo_inertia[:3, 3] = sp.Matrix(h)
    pseudo_inertia[3, :3] = sp.Matrix(h).T
    pseudo_inertia[3, 3] = m

    return pseudo_inertia

Thus we may get the pseudo inertia matrix as:

In [ ]:
pseudo = theta2pseudo(params)
pseudo

Matrix([
[-0.5*I_xx + 0.5*I_yy + 0.5*I_zz,                          -I_xy,                          -I_xz, mr_x],
[                          -I_xy, 0.5*I_xx - 0.5*I_yy + 0.5*I_zz,                          -I_yz, mr_y],
[                          -I_xz,                          -I_yz, 0.5*I_xx + 0.5*I_yy - 0.5*I_zz, mr_z],
[                           mr_x,                           mr_y,                           mr_z,    m]])

In [ ]:
print(pseudo)

Matrix([[-0.5*I_xx + 0.5*I_yy + 0.5*I_zz, -I_xy, -I_xz, mr_x], [-I_xy, 0.5*I_xx - 0.5*I_yy + 0.5*I_zz, -I_yz, mr_y], [-I_xz, -I_yz, 0.5*I_xx + 0.5*I_yy - 0.5*I_zz, mr_z], [mr_x, mr_y, mr_z, m]])


Conversion back is done as simple as:

In [ ]:
def pseudo2theta(pseudo_inertia):
    """
    Converts a pseudo inertia matrix back to theta parameters using sympy.

    Args:
        pseudo_inertia (sp.Matrix): Pseudo inertia matrix.

    Returns:
        sp.Matrix: Vector of parameters [m, h_x, h_y, h_z, I_xx, I_xy, I_yy, I_xz, I_yz, I_zz].
    """
    # Extract mass and first moments
    m = pseudo_inertia[3, 3]
    h = pseudo_inertia[:3, 3]

    # Extract and compute the Sigma matrix
    Sigma = pseudo_inertia[:3, :3]

    # Compute the inertia tensor I_bar from Sigma
    trace_Sigma = sp.trace(Sigma)
    I_bar = trace_Sigma * sp.eye(3) -  Sigma

    # Extract individual components of the inertia tensor
    I_xx = I_bar[0, 0]
    I_xy = I_bar[0, 1]
    I_yy = I_bar[1, 1]
    I_xz = I_bar[0, 2]
    I_yz = I_bar[1, 2]
    I_zz = I_bar[2, 2]

    # Create a vector of the theta parameters
    theta = sp.Matrix([m, h[0], h[1], h[2], I_xx, I_xy, I_yy, I_xz, I_yz, I_zz])

    return theta



In [ ]:
pseudo2theta(pseudo)

Matrix([
[       m],
[    mr_x],
[    mr_y],
[    mr_z],
[1.0*I_xx],
[    I_xy],
[1.0*I_yy],
[    I_xz],
[    I_yz],
[1.0*I_zz]])


This matrix can also be formulated through a density function using homogeneous coordinates:

$$
\mathbf{q} = \left[\begin{array}{c}
\mathbf{x} \\
1
\end{array}\right]
$$

The pseudo-inertia is expressed as:

$$
\mathbf{J} = \int_{V} \mathbf{q} \mathbf{q}^\top \rho(\mathbf{x}) \mathrm{d} V
$$

---

It has been established that $\theta$ is fully physically consistent if and only if:

$$\mathbf{J}(\boldsymbol{\theta}) \succ 0$$

---

This finding is crucial in expressing physical consistency through linear matrix inequalities and in recognizing that physical consistency is equivalent to the Riemannian manifold of symmetric positive definite $4 \times 4$ matrices: $\mathcal{P}^{4}$. This geometric interpretation immediately introduces the exponential/logarithmic map parametrization, which utilizes matrix exponentials and logarithms:

$$
\exp (\mathbf{T}) = \sum_{k=0}^{\infty} \frac{\mathbf{T}^k}{k!} = \mathbf{P} \in \mathcal{P}^{4}
$$

$$
\log (\mathbf{P}) = -\sum_{k=1}^{\infty} \frac{(\mathbf{1}-\mathbf{P})^k}{k} = \mathbf{T} \in \mathcal{S}^{4}
$$

Taking the upper triangular elements of the matrix logarithm provides the following parametrization:

$$
\boldsymbol{\gamma} = \operatorname{vech}\{\log (\mathbf{J}(\boldsymbol{\theta}))\}
$$

and the parameters are reconstructed by:

$$
\mathbf{J}(\boldsymbol{\theta}) = \exp (\operatorname{invvech}(\boldsymbol{\gamma}))
$$

where the inverse vectorization operator $\operatorname{invvech}$ constructs the symmetric matrix from its upper triangular elements.

This parametrization is both practically and theoretically intriguing, offering insights into the geometry of inertia, clearly demonstrating that it does not conform to the vector space $\mathbb{R}^{10}$. However, the physical meaning of this parameterization is not immediately apparent, and its computational complexity does not allow for a straightforward formulation of the Jacobian. This complexity has led to the development of the novel Log-Cholesky parametrization, which simplifies some of these challenges.

### **Log-Cholesky**

In the context of ensuring physical consistency constraints, the requirement for the positive definiteness of pseudo inertia is critical. Recent research by Rucker et al. proposes the use of Log Cholesky decomposition as a novel method to explicitly parameterize the pseudo inertia in terms of elements of the decomposed matrix. This approach leverages the decomposition of a real symmetric matrix \(\mathbf{J}\), which can be elegantly expressed using the Cholesky decomposition:

$$
\mathbf{J} = \mathbf{U} \mathbf{U}^{\top}
$$

Here,$\mathbf{U}$ is an upper triangular matrix with positive diagonal elements, though the decomposition could alternatively utilize a lower triangular matrix.

An essential feature of upper triangular matrices, like $\mathbf{U}$, is their nonsingularity, contingent upon all diagonal elements being nonzero. This characteristic can be captured by parametrizing the elements of the Cholesky decomposition as follows:

$$
\mathbf{U} = e^{\alpha} \left[\begin{array}{cccc}
e^{d_1} & s_{12} & s_{13} & t_1 \\
0 & e^{d_2} & s_{23} & t_2 \\
0 & 0 & e^{d_3} & t_3 \\
0 & 0 & 0 & 1
\end{array}\right]
$$

This formulation reflects a variant of the [Log-Cholesky decomposition](https://arxiv.org/pdf/1908.09326), where the bottom-right scalar term is explicitly factored out.


Let us create the routines that will convert the Log-Cholesky parameters top its matrix representation:

In [ ]:
def logchol2chol(phi):
    """
    Constructs the matrix U from the given parameters.

    Args:
        phi (list or sympy.Matrix): Contains the parameters:
            [alpha, d1, d2, d3, s12, s23, s13, t1, t2, t3]

    Returns:
        sympy.Matrix: The matrix U.
    """
    # Unpack the parameters from the input vector
    alpha, d1, d2, d3, s12, s23, s13, t1, t2, t3 = phi

    # Compute the exponential terms
    exp_alpha = sp.exp(alpha)
    exp_d1 = sp.exp(d1)
    exp_d2 = sp.exp(d2)
    exp_d3 = sp.exp(d3)

    # Create the matrix U
    U = sp.Matrix([
        [exp_d1, s12, s13, t1],
        [0, exp_d2, s23, t2],
        [0, 0, exp_d3, t3],
        [0, 0, 0, 1]
    ])

    # Multiply the entire matrix by exp(alpha)
    U = exp_alpha * U

    return U



---

The Log-Cholesky parameters can be defined as a vector in $\mathbb{R}^{10}$:

$$
\boldsymbol{\phi} = \left[\alpha, d_1, d_2, d_3, s_{12}, s_{23}, s_{13}, t_1, t_2, t_3\right]^{\top}
$$

---


In [ ]:
phi = sp.Matrix(sp.symbols('alpha d_1 d_2 d_3 s_12 s_23 s_13 t_1 t_2 t_3'))
phi

Matrix([
[alpha],
[  d_1],
[  d_2],
[  d_3],
[ s_12],
[ s_23],
[ s_13],
[  t_1],
[  t_2],
[  t_3]])

Let us build the upper triangular matrix from above parameters:

In [ ]:
U = logchol2chol(phi)
U

Matrix([
[exp(alpha)*exp(d_1),     s_12*exp(alpha),     s_13*exp(alpha), t_1*exp(alpha)],
[                  0, exp(alpha)*exp(d_2),     s_23*exp(alpha), t_2*exp(alpha)],
[                  0,                   0, exp(alpha)*exp(d_3), t_3*exp(alpha)],
[                  0,                   0,                   0,     exp(alpha)]])

Using these parameters in the Cholesky decomposition, we compute the matrix \(\mathbf{J}\):

$$
\mathbf{J}(\boldsymbol{\phi}) = \mathbf{U}(\boldsymbol{\phi}) \mathbf{U}^{\top}(\boldsymbol{\phi})
$$

The design of $\boldsymbol{\phi}$ ensures that the resulting pseudo inertia $\mathbf{J}$ is always positive definite, thereby satisfying physical consistency without imposing constraints on $\boldsymbol{\phi}$.



In [ ]:
pseudo_cholesky = U@U.T

In [ ]:
pseudo_cholesky

Matrix([
[s_12**2*exp(2*alpha) + s_13**2*exp(2*alpha) + t_1**2*exp(2*alpha) + exp(2*alpha)*exp(2*d_1), s_12*exp(2*alpha)*exp(d_2) + s_13*s_23*exp(2*alpha) + t_1*t_2*exp(2*alpha), s_13*exp(2*alpha)*exp(d_3) + t_1*t_3*exp(2*alpha), t_1*exp(2*alpha)],
[                 s_12*exp(2*alpha)*exp(d_2) + s_13*s_23*exp(2*alpha) + t_1*t_2*exp(2*alpha),       s_23**2*exp(2*alpha) + t_2**2*exp(2*alpha) + exp(2*alpha)*exp(2*d_2), s_23*exp(2*alpha)*exp(d_3) + t_2*t_3*exp(2*alpha), t_2*exp(2*alpha)],
[                                          s_13*exp(2*alpha)*exp(d_3) + t_1*t_3*exp(2*alpha),                          s_23*exp(2*alpha)*exp(d_3) + t_2*t_3*exp(2*alpha),     t_3**2*exp(2*alpha) + exp(2*alpha)*exp(2*d_3), t_3*exp(2*alpha)],
[                                                                           t_1*exp(2*alpha),                                                           t_2*exp(2*alpha),                                  t_3*exp(2*alpha),     exp(2*alpha)]])

In [ ]:
print(pseudo_cholesky)

Matrix([[s_12**2*exp(2*alpha) + s_13**2*exp(2*alpha) + t_1**2*exp(2*alpha) + exp(2*alpha)*exp(2*d_1), s_12*exp(2*alpha)*exp(d_2) + s_13*s_23*exp(2*alpha) + t_1*t_2*exp(2*alpha), s_13*exp(2*alpha)*exp(d_3) + t_1*t_3*exp(2*alpha), t_1*exp(2*alpha)], [s_12*exp(2*alpha)*exp(d_2) + s_13*s_23*exp(2*alpha) + t_1*t_2*exp(2*alpha), s_23**2*exp(2*alpha) + t_2**2*exp(2*alpha) + exp(2*alpha)*exp(2*d_2), s_23*exp(2*alpha)*exp(d_3) + t_2*t_3*exp(2*alpha), t_2*exp(2*alpha)], [s_13*exp(2*alpha)*exp(d_3) + t_1*t_3*exp(2*alpha), s_23*exp(2*alpha)*exp(d_3) + t_2*t_3*exp(2*alpha), t_3**2*exp(2*alpha) + exp(2*alpha)*exp(2*d_3), t_3*exp(2*alpha)], [t_1*exp(2*alpha), t_2*exp(2*alpha), t_3*exp(2*alpha), exp(2*alpha)]])


**Transformation to Inertial Parameters**

The transformation of elements from $\mathbf{J}$ to inertial parameters $\boldsymbol{\theta}$ is given by:

In [ ]:
theta_logchol = pseudo2theta(pseudo_cholesky)
theta_logchol

Matrix([
[                                                                                                                                                      exp(2*alpha)],
[                                                                                                                                                  t_1*exp(2*alpha)],
[                                                                                                                                                  t_2*exp(2*alpha)],
[                                                                                                                                                  t_3*exp(2*alpha)],
[                                              s_23**2*exp(2*alpha) + t_2**2*exp(2*alpha) + t_3**2*exp(2*alpha) + exp(2*alpha)*exp(2*d_2) + exp(2*alpha)*exp(2*d_3)],
[                                                                                       -s_12*exp(2*alpha)*exp(d_2) - s_13*s_23*exp(2*alpha) - t_1*t_2*exp(2*alph

In [ ]:
print(theta_logchol)

Matrix([[exp(2*alpha)], [t_1*exp(2*alpha)], [t_2*exp(2*alpha)], [t_3*exp(2*alpha)], [s_23**2*exp(2*alpha) + t_2**2*exp(2*alpha) + t_3**2*exp(2*alpha) + exp(2*alpha)*exp(2*d_2) + exp(2*alpha)*exp(2*d_3)], [-s_12*exp(2*alpha)*exp(d_2) - s_13*s_23*exp(2*alpha) - t_1*t_2*exp(2*alpha)], [s_12**2*exp(2*alpha) + s_13**2*exp(2*alpha) + t_1**2*exp(2*alpha) + t_3**2*exp(2*alpha) + exp(2*alpha)*exp(2*d_1) + exp(2*alpha)*exp(2*d_3)], [-s_13*exp(2*alpha)*exp(d_3) - t_1*t_3*exp(2*alpha)], [-s_23*exp(2*alpha)*exp(d_3) - t_2*t_3*exp(2*alpha)], [s_12**2*exp(2*alpha) + s_13**2*exp(2*alpha) + s_23**2*exp(2*alpha) + t_1**2*exp(2*alpha) + t_2**2*exp(2*alpha) + exp(2*alpha)*exp(2*d_1) + exp(2*alpha)*exp(2*d_2)]])


The Jacobian of this mapping, denoted as $\boldsymbol{J}_{\phi}$:

$$\boldsymbol{J}_\phi = \frac{\partial \theta}{\partial \phi}$$

In [ ]:
theta_logchol.jacobian(phi)

Matrix([
[                                                                                                                                                                  2*exp(2*alpha),                         0,                           0,                           0,                      0,                      0,                      0,                  0,                  0,                  0],
[                                                                                                                                                              2*t_1*exp(2*alpha),                         0,                           0,                           0,                      0,                      0,                      0,       exp(2*alpha),                  0,                  0],
[                                                                                                                                                              2*t_2*exp(2*alpha),                   

the only non-constant factors of the determinant of $\boldsymbol{J}_\phi$ are $e^{\alpha}, e^d_{1}, e^{d_{2}}, e^{d_{3}}$:

In [ ]:
sp.det(theta_logchol.jacobian(phi))

32*exp(20*alpha)*exp(2*d_1)*exp(3*d_2)*exp(4*d_3)

Thus mapping $\boldsymbol{\theta}(\boldsymbol{\phi})$ smooth and invertable (diffeomorphism) and does not introduce new local minima, making it particularly effective for adaptation and optimization contexts where physical constraints are paramount.

## Stay Tuned

This is just the beginning. Soon we will provide more theory on some related topics, mainly on:

* [Riemannian regularization](https://ieeexplore.ieee.org/document/8922724)
* [Alternative perspectives on regressor models: energy, power, momentum](https://ieeexplore.ieee.org/document/574537)